### 一、循环神经网络RNN
#### 1. RNN结构
   &ensp;&ensp;&ensp;&ensp;单层RNN基本结构如下，输入一个序列 $x_{0}$,$x_{1}$,...,$x_{t}$，产生输出序列 $h_{0}$,$h_{1}$,...,$h_{t}$和最后一个时刻隐状态，但输入权重、隐藏层输出权重以及激活函数均不变:
   ![rnn_struct](http://localhost:9999/files/pic/rnn_struct.jpg)
   &ensp;&ensp;&ensp;&ensp;RNN具体结构如下：
   ![](http://localhost:9999/files/pic/rnn_detail.jpg)
   对输入序列中每个元素，RNN每层的计算公式为:
   $$ h_t = tanh(w_{ih} x_t + b_{ih} + w_{hh} h_{t-1} + b_{hh}) $$
   $h_{t-1}$是时刻$t-1$的隐状态。 $x_t$是上一层时刻$t$的隐状态，或者是第一层在时刻$t$的输入。
   Pytoch最后只输出隐状态$h_t$，若要输出$y_t$，则需要根据隐状态计算输出状态:
   $$ y_t = softmax(w_{hy} h_t + b_{hy}) $$
#### 2. Pytoch实现RNN
   &ensp;&ensp;&ensp;&ensp;Pytoch中RNN实现类: 
   >torch.nn.RNN( args, * kwargs)
   参数如下：
   * input_size – 输入x的特征数量。
   * hidden_size – 隐层的特征数量。
   * num_layers – RNN的层数。
   * nonlinearity – 指定非线性函数使用tanh还是relu。默认是tanh。
   * bias – 如果是False，那么RNN层就不会使用偏置权重 $b_ih$和$b_hh$，默认是True
   * batch_first – 如果True的话，那么输入Tensor的shape应该是[batch_size, time_step, feature]，输出也是这样。
   * dropout – 如果值非零，那么除了最后一层外，其它层的输出都会套上一个dropout层。
   * bidirectional – 如果True，将会变成一个双向RNN，默认为False。
 
##### 2.1 定义输入 $x$和初始隐状态 $h_0$
&ensp;&ensp;&ensp;&ensp;假设定义一个单层非循环的RNN结构，输入特征大小为2，输出隐状态的特征大小为4。
 * RNN结构: 输入特征大小为2，隐状态特征大小为4,单层结构。
 > rnn = nn.RNN(2, 4, 1)
 
 * 输入数据 $x$(seq_len, batch, input_size): 特征向量大小为2，样本只有1个批次，输入序列长度为3。
 > x = Variable(torch.randn(3, 1, 2))
 
 * 初始隐状态 $h_0$ (num_layers \* num_directions, batch, hidden_size): 单层非循环，样本只有1个批次，隐状态的特征大小为4。
 > h0 = Variable(torch.randn(1\*1, 1, 4))

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
#定义RNN
rnn = nn.RNN(2, 4, 1)
#输入数据
x = Variable(torch.randn(3, 1, 2))
#初始隐状态
h0 = Variable(torch.randn(1*1, 1, 4))

#### 2.2 RNN模型权重
定义好RNN后会初始化模型参数

**RNN模型参数:**
* weight_ih_l[k] – 第k层的 input-hidden 权重， 可学习，维度是(hidden_size x input_size)。
* weight_hh_l[k] – 第k层的 hidden-hidden 权重， 可学习，维度是(hidden_size x hidden_size)
* bias_ih_l[k] – 第k层的 input-hidden 偏置， 可学习，维度是(hidden_size)
* bias_hh_l[k] – 第k层的 hidden-hidden 偏置， 可学习，维度是(hidden_size)


In [52]:
#weight_ih_l[k] 
weight_ih_l0 = rnn.weight_ih_l0
#bias_ih_l[k]
bias_ih_l0 = rnn.bias_ih_l0
#weight_hh_l[k]
weight_hh_l0 = rnn.weight_hh_l0
#bias_hh_l[k]
bias_hh_l0 = rnn.bias_hh_l0
#模型参数状态
print 'weight_ih_l[k]维度:',weight_ih_l0.size(),'\n',\
      'bias_ih_l[k]维度:',bias_ih_l0.size(),'\n',\
      'weight_hh_l[k]维度:',weight_hh_l0.size(),'\n',\
      'bias_hh_l[k]维度:',bias_hh_l0.size()

weight_ih_l[k]维度: torch.Size([4, 2]) 
bias_ih_l[k]维度: torch.Size([4]) 
weight_hh_l[k]维度: torch.Size([4, 4]) 
bias_hh_l[k]维度: torch.Size([4])


#### 2.3 RNN输出
**RNN的输出： (output, h_n)**

* output (seq_len, batch, hidden_size * num_directions): 保存着RNN最后一层的输出特征。如果输入是被填充过的序列，那么输出也是被填充的序列。
* h_n (num_layers * num_directions, batch, hidden_size): 保存着最后一个时刻隐状态。

In [51]:
#RNN输出
output, h_n = rnn(x, h0)
#output和最后1个时刻隐状态hn
print '输出output的维度:',output.size(),'\n',\
      '隐状态h_n的维度:', h_n.size()

输出output的维度: torch.Size([3, 1, 4]) 
隐状态h_n的维度: torch.Size([1, 1, 4])


#### 2.4 自己根据RNN公式计算输出和隐状态
**(1) 首先定义一个函数来判断两个向量是否相等**<br/>
由于向量元素是float类型，所以元素相差1e-6，就可认为两个元素相等。
>def is_equal(v1,v2):<br/>

**(2) RNN计算流程**
 ![](http://localhost:9999/files/pic/rnn_compute.jpg)
* 第1个输入$x_0$和隐状态$h_0$，输出$h_1$，其中$h_1$和$out_1$相同。
> x_0 = x[0,0,:].unsqueeze(1) <br/>
  h_0 = h0[0,0,:].unsqueeze(1)<br/>
h_1 = F.tanh(weight_ih_l0.mm(x_0)+bias_ih_l0+weight_hh_l0.mm(h_0)+bias_hh_l0)<br/>
is_equal(h_1,output[0,0,:])

* 第2个输入$x_1$和$h_1$，输出$h_2$，其中$h_2$和$out_2$相同。
> x_1 = x[1,0,:].unsqueeze(1) <br/>
  h_2 = F.tanh(weight_ih_l0.mm(x_1)+bias_ih_l0+weight_hh_l0.mm(h_1)+bias_hh_l0) <br/>
  is_equal(h_2,output[1,0,:])

* 第3个输入$x_2$和$h_2$，输出$h_3$，其中$h_3$和$out_3$相同。
> x_2 = x[2,0,:].unsqueeze(1) <br/>
h_3 = F.tanh(weight_ih_l0.mm(x_2)+bias_ih_l0+weight_hh_l0.mm(h_2)+bias_hh_l0) <br/>
is_equal(h_3,output[2,0,:])

* RNN最后一个时刻隐状态$h_n$和第3个隐状态$h_3$相同。
> is_equal(h_n,h_3)

In [48]:
#判断两个Variable向量是否相等
def is_equal(v1,v2):
    diff = torch.abs(v1-v2)
    diff.sum
    eq=(diff<=1e-6).sum().data[0]
    if eq<v1.size(0):
        return False
    else:
        return True 

In [49]:
#第0个输入,并将列表转为2*1
x_0 = x[0,0,:].unsqueeze(1)
#第0个隐状态
h_0 = h0[0,0,:].unsqueeze(1)
#第1个隐状态
h_1 = F.tanh(weight_ih_l0.mm(x_0)+bias_ih_l0+weight_hh_l0.mm(h_0)+bias_hh_l0)
#判断第1个输出和第1个隐状态是否相等
print '第1个隐状态和第1个输出是否相等:',is_equal(h_1,output[0,0,:])

#第1个输入,并将列表转为2*1
x_1 = x[1,0,:].unsqueeze(1)
#第2个隐状态
h_2 = F.tanh(weight_ih_l0.mm(x_1)+bias_ih_l0+weight_hh_l0.mm(h_1)+bias_hh_l0)
#判断第1个输出和第1个隐状态是否相等
print '第2个隐状态和第2个输出是否相等:',is_equal(h_2,output[1,0,:])

#第2个输入,并将列表转为2*1
x_2 = x[2,0,:].unsqueeze(1)
#第3个隐状态
h_3 = F.tanh(weight_ih_l0.mm(x_2)+bias_ih_l0+weight_hh_l0.mm(h_2)+bias_hh_l0)
#判断第1个输出和第1个隐状态是否相等
print '第3个隐状态和第3个输出是否相等:',is_equal(h_3,output[2,0,:])

#判断RNN最后一个时刻隐状态和第3个隐状态是否相等
print 'RNN最后一个时刻隐状态和第3个隐状态是否相等:',is_equal(h_n,h_3)

第1个隐状态和第1个输出是否相等: True
第2个隐状态和第2个输出是否相等: True
第3个隐状态和第3个输出是否相等: True
RNN最后一个时刻隐状态和第3个隐状态是否相等: True


#### 3. 单向双层RNN
单向双层RNN的结构如下：
![rnn_muti](http://localhost:9999/files/pic/rnn_muti.jpg)
#### 3.1 定义单向双层RNN
定义RNN隐藏层层数的参数是num_layers。
下面定义一个num_layers是2的单向双层RNN。
* 单向双层RNN结构(input_size,hidden_size,num_layers)：输入特征大小为2，隐状态特征大小为4,隐藏层层数为2。
> rnn_two = RNN(2,4,2)

* 输入数据 x (seq_len, batch, input_size): 特征向量大小为2，样本只有1个批次，输入序列长度为3。
> x_two = Variable(torch.randn(3, 1, 2))

* 初始隐状态 $h_0$ (num_layers * num_directions, batch, hidden_size): 双层单向，样本只有1个批次，隐状态的特征大小为4。
> h0_two = Variable(torch.randn(2\*1, 1, 4))

In [31]:
#定义RNN
rnn_two = nn.RNN(2, 4, 2)
#输入数据(seq_len, batch, input_size)
x_two = Variable(torch.randn(3, 1, 2))
#初始隐状态(num_layers * num_directions, batch, hidden_size)
h0_two = Variable(torch.randn(2*1, 1, 4))

#### 3.2 RNN模型权重
定义好RNN后会初始化模型参数。

**(1) 第1层隐藏层模型参数:**
* weight_ih_l0 – 第1层的 input-hidden 权重， 可学习，维度是(hidden_size x input_size)，即(4 x 2)。
* weight_hh_l0 – 第1层的 hidden-hidden 权重， 可学习，维度是(hidden_size x hidden_size)，即(4 x 4)。
* bias_ih_l0 – 第1层的 input-hidden 偏置， 可学习，维度是(hidden_size)，即(4)。
* bias_hh_l0 – 第1层的 hidden-hidden 偏置， 可学习，维度是(hidden_size)，即(4)。

**(2) 第2层隐藏层模型参数:**
* weight_ih_l1 – 第2层的 input-hidden 权重， 可学习，维度是(hidden_size x input_size)，即(4 x 4)。
* weight_hh_l1 – 第2层的 hidden-hidden 权重， 可学习，维度是(hidden_size x hidden_size)，即(4 x 4)。
* bias_ih_l1 – 第2层的 input-hidden 偏置， 可学习，维度是(hidden_size)，即(4)。
* bias_hh_l1 – 第2层的 hidden-hidden 偏置， 可学习，维度是(hidden_size)，即(4)。

In [55]:
#第1层隐藏层模型参数
# input-hidden 权重
weight_ih_l0 = rnn_two.weight_ih_l0
#input-hidden 偏置
bias_ih_l0 = rnn_two.bias_ih_l0
#hidden-hidden 权重
weight_hh_l0 = rnn_two.weight_hh_l0
#hidden-hidden 偏置
bias_hh_l0 = rnn_two.bias_hh_l0

#模型参数状态
print '****第1层隐藏层模型参数维度****'
print 'weight_ih_l0维度:',weight_ih_l0.size(),'\n',\
      'bias_ih_l0维度:',bias_ih_l0.size(),'\n',\
      'weight_hh_l0维度:',weight_hh_l0.size(),'\n',\
      'bias_hh_l0维度:',bias_hh_l0.size()

#第2层隐藏层模型参数
# input-hidden 权重
weight_ih_l1 = rnn_two.weight_ih_l1
#input-hidden 偏置
bias_ih_l1 = rnn_two.bias_ih_l1
#hidden-hidden 权重
weight_hh_l1 = rnn_two.weight_hh_l1
#hidden-hidden 偏置
bias_hh_l1 = rnn_two.bias_hh_l1

#模型参数状态
print '****第2层隐藏层模型参数维度****'
print 'weight_ih_l1维度:',weight_ih_l1.size(),'\n',\
      'bias_ih_l1维度:',bias_ih_l1.size(),'\n',\
      'weight_hh_l1维度:',weight_hh_l1.size(),'\n',\
      'bias_hh_l1维度:',bias_hh_l1.size()

****第1层隐藏层模型参数维度****
weight_ih_l0维度: torch.Size([4, 2]) 
bias_ih_l0维度: torch.Size([4]) 
weight_hh_l0维度: torch.Size([4, 4]) 
bias_hh_l0维度: torch.Size([4])
****第2层隐藏层模型参数维度****
weight_ih_l1维度: torch.Size([4, 4]) 
bias_ih_l1维度: torch.Size([4]) 
weight_hh_l1维度: torch.Size([4, 4]) 
bias_hh_l1维度: torch.Size([4])


#### 3.3 RNN输出
**RNN的输出： (output, h_n)**

* output (seq_len, batch, hidden_size * num_directions): 保存着RNN最后一层的输出特征。如果输入是被填充过的序列，那么输出也是被填充的序列。
* h_n (num_layers * num_directions, batch, hidden_size): 保存着最后一个时刻隐状态。

In [56]:
#单向双层RNN输出
output, h_n = rnn_two(x_two, h0_two)
#output和最后1个时刻隐状态hn
print '输出output的维度:',output.size(),'\n',\
      '隐状态h_n的维度:', h_n.size()

输出output的维度: torch.Size([3, 1, 4]) 
隐状态h_n的维度: torch.Size([2, 1, 4])


#### 3.4 自己根据RNN公式计算输出和隐状态
**(1) 单向双层RNN计算流程**
![](http://localhost:9999/files/pic/rnn_two.jpg)
**[1]第1层隐藏层计算**
* 第1个输入$x_0$和初始隐状态$h_{10}$，输出$h_{11}$。
> x_0 = x_two[0,0,:].unsqueeze(1) <br/>
  h1_0 = h0_two[0,0,:].unsqueeze(1)<br/>
h1_1 = F.tanh(weight_ih_l0.mm(x_0)+bias_ih_l0+weight_hh_l0.mm(h1_0)+bias_hh_l0)<br/>

* 第2个输入$x_1$和隐状态$h_{11}$，输出$h_{12}$。
> x_1 = x_two[1,0,:].unsqueeze(1) <br/>
h1_2 = F.tanh(weight_ih_l0.mm(x_1)+bias_ih_l0+weight_hh_l0.mm(h1_1)+bias_hh_l0)<br/>

* 第3个输入$x_2$和隐状态$h_{12}$，输出$h_{13}$，$h_{13}$和$hn_1$相同。
> x_2 = x_two[2,0,:].unsqueeze(1) <br/>
h1_3 = F.tanh(weight_ih_l0.mm(x_2)+bias_ih_l0+weight_hh_l0.mm(h1_2)+bias_hh_l0)<br/>
is_equal(h1_3,h_n[0,0,:])

**[1]第2层隐藏层计算**
* 第1个输入$h_{10}$和初始隐状态$h_{20}$，输出$h_{21}$，$h_{21}$和$out_1$相同。
> h2_0 = h0_two[1,0,:].unsqueeze(1)<br/>
h2_1 = F.tanh(weight_ih_l1.mm(h1_1)+bias_ih_l1+weight_hh_l1.mm(h2_0)+bias_hh_l1)<br/>
is_equal(h2_1,output[0,0,:])

* 第2个输入$h_{11}$和隐状态$h_{21}$，输出$h_{22}$，$h_{22}$和$out_2$相同。
> h2_2 = F.tanh(weight_ih_l1.mm(h1_2)+bias_ih_l1+weight_hh_l1.mm(h2_1)+bias_hh_l1)<br/>
is_equal(h2_2,output[1,0,:])

* 第3个输入$h_{12}$和隐状态$h_{22}$，输出$h_{23}$，$h_{23}$和$out_3$相同，也和$hn_2$相同。
> h2_3 = F.tanh(weight_ih_l1.mm(h1_3)+bias_ih_l1+weight_hh_l1.mm(h2_2)+bias_hh_l1)<br/>
is_equal(h2_3,output[2,0,:])<br/>
is_equal(h2_3,h_n[1,0,:])

In [65]:
#第1层隐藏层计算
#第1个输入x_0和初始隐状态h1_0，输出隐状态h1_1
x_0 = x_two[0,0,:].unsqueeze(1) 
h1_0 = h0_two[0,0,:].unsqueeze(1)
h1_1 = F.tanh(weight_ih_l0.mm(x_0)+bias_ih_l0+weight_hh_l0.mm(h1_0)+bias_hh_l0)

#第2个输入x_1和h1_1，输出隐状态h1_2
x_1 = x_two[1,0,:].unsqueeze(1) 
h1_2 = F.tanh(weight_ih_l0.mm(x_1)+bias_ih_l0+weight_hh_l0.mm(h1_1)+bias_hh_l0)

#第3个输入x_2和h1_2，输出隐状态h1_3
x_2 = x_two[2,0,:].unsqueeze(1) 
h1_3 = F.tanh(weight_ih_l0.mm(x_2)+bias_ih_l0+weight_hh_l0.mm(h1_2)+bias_hh_l0)
#判断h1_3和hn_1是否相等
print '第1层最后1个隐状态和第1个输出隐状态是否相等:',is_equal(h1_3,h_n[0,0,:])

#第2层隐藏层计算
#第1个输入h1_1和初始隐状态h2_0
h2_0 = h0_two[1,0,:].unsqueeze(1)
h2_1 = F.tanh(weight_ih_l1.mm(h1_1)+bias_ih_l1+weight_hh_l1.mm(h2_0)+bias_hh_l1)
#判断第2层第1个隐状态是否等于第1个输出
print '第2层第1个隐状态是否等于第1个输出:',is_equal(h2_1,output[0,0,:])

#第2个输入h1_2和上个隐状态h2_1
h2_2 = F.tanh(weight_ih_l1.mm(h1_2)+bias_ih_l1+weight_hh_l1.mm(h2_1)+bias_hh_l1)
#判断第2层第2个隐状态是否等于第3个输出
print '第2层第2个隐状态是否等于第2个输出:',is_equal(h2_2,output[1,0,:])

#第3个输入h1_3和上个隐状态h2_2
h2_3 = F.tanh(weight_ih_l1.mm(h1_3)+bias_ih_l1+weight_hh_l1.mm(h2_2)+bias_hh_l1)
#判断第2层第3个隐状态是否等于第3个输出
print '第2层第3个隐状态是否等于第3个输出:',is_equal(h2_3,output[2,0,:])
#判断第2层第3个隐状态是否等于第2个输出隐状态
print '第2层第3个隐状态是否等于第2个输出隐状态',is_equal(h2_3,h_n[1,0,:])

第1层最后1个隐状态和第1个输出隐状态是否相等: True
第2层第1个隐状态是否等于第1个输出: True
第2层第2个隐状态是否等于第2个输出: True
第2层第3个隐状态是否等于第3个输出: True
第2层第3个隐状态是否等于第2个输出隐状态 True


####  4. 双向双层RNN
双向双层RNN的结构如下:
![rnn_bi_muti](http://localhost:9999/files/pic/rnn_bi_muti.jpg)
定义双向RNN的参数是bidirectional,如果bidirectional为True,则是双向,默认为False。
下面定义一个双向双层RNN。
* 双向双层RNN结构(input_size,hidden_size,num_layers,bidirectional=True)：输入特征大小为2，隐状态特征大小为4,隐藏层层数为2。
> rnn_bi_two = RNN(2,4,2,bidirectional=True)
* 输入数据 x (seq_len, batch, input_size): 特征向量大小为2，样本只有1个批次，输入序列长度为3。
> x_bi_two = Variable(torch.randn(3, 1, 2))
* 初始隐状态 $h_0$ (num_layers * num_directions, batch, hidden_size): 双向双层，样本只有1个批次，隐状态的特征大小为4。
> h0_bi_two = Variable(torch.randn(2\*2, 1, 4))

In [2]:
#定义RNN
rnn_bi_two = nn.RNN(2, 4, 2, bidirectional=True)
#输入数据(seq_len, batch, input_size)
x_bi_two = Variable(torch.randn(3, 1, 2))
#初始隐状态(num_layers * num_directions, batch, hidden_size)
h0_bi_two = Variable(torch.randn(2*2, 1, 4))

#### 3.2 RNN模型权重
定义好RNN后会初始化模型参数。

**(1) 第1层隐藏层模型顺序参数:**
* weight_ih_l0 – 第1层的 input-hidden 顺序权重， 可学习，维度是(hidden_size x input_size)，即(4 x 2)。
* weight_hh_l0 – 第1层的 hidden-hidden 顺序权重， 可学习，维度是(hidden_size x hidden_size)，即(4 x 4)。
* bias_ih_l0 – 第1层的 input-hidden 顺序偏置， 可学习，维度是(hidden_size)，即(4)。
* bias_hh_l0 – 第1层的 hidden-hidden 顺序偏置， 可学习，维度是(hidden_size)，即(4)。

**(2) 第1层隐藏层模型逆序参数:**
* weight_ih_l0_reverse – 第1层的 input-hidden 逆序权重， 可学习，维度是(hidden_size x input_size)，即(4 x 2)。
* weight_hh_l0_reverse – 第1层的 hidden-hidden 逆序权重， 可学习，维度是(hidden_size x hidden_size)，即(4 x 4)。
* bias_ih_l0_reverse – 第1层的 input-hidden 逆序偏置， 可学习，维度是(hidden_size)，即(4)。
* bias_hh_l0_reverse – 第1层的 hidden-hidden 逆序偏置， 可学习，维度是(hidden_size)，即(4)。

**(3) 第2层隐藏层模型顺序参数:**
上层顺序和逆序的输出大小都为4，双向循环会将两个输出结果横向合并作为该层的输入,因此输入大小input_size变为8.
* weight_ih_l1 – 第1层的 input-hidden 顺序权重， 可学习，维度是(hidden_size x input_size)，即(4 x 8)。
* weight_hh_l1 – 第1层的 hidden-hidden 顺序权重， 可学习，维度是(hidden_size x hidden_size)，即(4 x 8)。
* bias_ih_l1 – 第1层的 input-hidden 顺序偏置， 可学习，维度是(hidden_size)，即(4)。
* bias_hh_l1 – 第1层的 hidden-hidden 顺序偏置， 可学习，维度是(hidden_size)，即(4)。

**(4) 第2层隐藏层模型逆序参数:**
* weight_ih_l1_reverse – 第1层的 input-hidden 逆序权重， 可学习，维度是(hidden_size x input_size)，即(4 x 8)。
* weight_hh_l1_reverse – 第1层的 hidden-hidden 逆序权重， 可学习，维度是(hidden_size x hidden_size)，即(4 x 4)。
* bias_ih_l1_reverse – 第1层的 input-hidden 逆序偏置， 可学习，维度是(hidden_size)，即(4)。
* bias_hh_l1_reverse – 第1层的 hidden-hidden 逆序偏置， 可学习，维度是(hidden_size)，即(4)。

In [6]:
#第1层隐藏层模型顺序参数
# input-hidden 顺序权重
weight_ih_l0 = rnn_bi_two.weight_ih_l0
#input-hidden 顺序偏置
bias_ih_l0 = rnn_bi_two.bias_ih_l0
#hidden-hidden 顺序权重
weight_hh_l0 = rnn_bi_two.weight_hh_l0
#hidden-hidden 顺序偏置
bias_hh_l0 = rnn_bi_two.bias_hh_l0

#模型参数状态
print '****第1层隐藏层模型顺序参数维度****'
print 'weight_ih_l0维度:',weight_ih_l0.size(),'\n',\
      'bias_ih_l0维度:',bias_ih_l0.size(),'\n',\
      'weight_hh_l0维度:',weight_hh_l0.size(),'\n',\
      'bias_hh_l0维度:',bias_hh_l0.size()

#第1层隐藏层模型逆序参数
# input-hidden 逆序权重
weight_ih_l0_reverse = rnn_bi_two.weight_ih_l0_reverse
#input-hidden 逆序偏置
bias_ih_l0_reverse = rnn_bi_two.bias_ih_l0_reverse
#hidden-hidden 逆序权重
weight_hh_l0_reverse = rnn_bi_two.weight_hh_l0_reverse
#hidden-hidden 逆序偏置
bias_hh_l0_reverse = rnn_bi_two.bias_hh_l0_reverse

#模型参数状态
print '****第1层隐藏层模型逆序参数维度****'
print 'weight_ih_l0_reverse维度:',weight_ih_l0_reverse.size(),'\n',\
      'bias_ih_l0_reverse维度:',bias_ih_l0_reverse.size(),'\n',\
      'weight_hh_l0_reverse维度:',weight_hh_l0_reverse.size(),'\n',\
      'bias_hh_l0_reverse维度:',bias_hh_l0_reverse.size()
    
#第2层隐藏层模型顺序参数
# input-hidden 顺序权重
weight_ih_l1 = rnn_bi_two.weight_ih_l1
#input-hidden 顺序偏置
bias_ih_l1 = rnn_bi_two.bias_ih_l1
#hidden-hidden 顺序权重
weight_hh_l1 = rnn_bi_two.weight_hh_l1
#hidden-hidden 顺序偏置
bias_hh_l1 = rnn_bi_two.bias_hh_l1

#模型参数状态
print '****第2层隐藏层模型顺序参数维度****'
print 'weight_ih_l1维度:',weight_ih_l1.size(),'\n',\
      'bias_ih_l1维度:',bias_ih_l1.size(),'\n',\
      'weight_hh_l1维度:',weight_hh_l1.size(),'\n',\
      'bias_hh_l1维度:',bias_hh_l1.size()

#第2层隐藏层模型逆序参数
# input-hidden 逆序权重
weight_ih_l1_reverse = rnn_bi_two.weight_ih_l1_reverse
#input-hidden 逆序偏置
bias_ih_l1_reverse = rnn_bi_two.bias_ih_l1_reverse
#hidden-hidden 逆序权重
weight_hh_l1_reverse = rnn_bi_two.weight_hh_l1_reverse
#hidden-hidden 逆序偏置
bias_hh_l1_reverse = rnn_bi_two.bias_hh_l1_reverse

#模型参数状态
print '****第1层隐藏层模型逆序参数维度****'
print 'weight_ih_l1_reverse维度:',weight_ih_l1_reverse.size(),'\n',\
      'bias_ih_l1_reverse维度:',bias_ih_l1_reverse.size(),'\n',\
      'weight_hh_l1_reverse维度:',weight_hh_l1_reverse.size(),'\n',\
      'bias_hh_l1_reverse维度:',bias_hh_l1_reverse.size()

****第1层隐藏层模型顺序参数维度****
weight_ih_l0维度: torch.Size([4, 2]) 
bias_ih_l0维度: torch.Size([4]) 
weight_hh_l0维度: torch.Size([4, 4]) 
bias_hh_l0维度: torch.Size([4])
****第1层隐藏层模型逆序参数维度****
weight_ih_l0_reverse维度: torch.Size([4, 2]) 
bias_ih_l0_reverse维度: torch.Size([4]) 
weight_hh_l0_reverse维度: torch.Size([4, 4]) 
bias_hh_l0_reverse维度: torch.Size([4])
****第2层隐藏层模型顺序参数维度****
weight_ih_l1维度: torch.Size([4, 8]) 
bias_ih_l1维度: torch.Size([4]) 
weight_hh_l1维度: torch.Size([4, 4]) 
bias_hh_l1维度: torch.Size([4])
****第1层隐藏层模型逆序参数维度****
weight_ih_l1_reverse维度: torch.Size([4, 8]) 
bias_ih_l1_reverse维度: torch.Size([4]) 
weight_hh_l1_reverse维度: torch.Size([4, 4]) 
bias_hh_l1_reverse维度: torch.Size([4])


In [8]:
output_bi_two, hn_bi_two = rnn_bi_two(x_bi_two, h0_bi_two)
#output和最后1个时刻隐状态hn
print '输出output的维度:',output_bi_two.size(),'\n',\
      '隐状态h_n的维度:', hn_bi_two.size()

输出output的维度: torch.Size([3, 1, 8]) 
隐状态h_n的维度: torch.Size([4, 1, 4])
